# Vstup a výstup

Načítání a ukládání dat z/na úložiště (*storage*) je moderními jazyky podporováno většinou přímo v základní syntaxi nebo knihovně funkcí. Jako programátoři se většinou nemusíme zabývat tím, jak jsou soubory s daty uloženy a jak uvnitř počítače funguje načítání a zápis. Nemusíme se starat o to, jak se čtou data z plotny pevného disku, síťového úložiště nebo optického média. Tuto službu nám poskytuje operační systém. V programovacím jazyce většinou jen nějakou funkcí "požádáme" o otevření souboru a potom z něj čteme nebo do něj zapisujeme pomocí dalších funkcí nebo metod (označení funkce navázané na určitý objekt). Schématicky práce se soubory vypadá nějak takto:

```
open file -> get filehandle
read/write from/to filehandle <-> data
close filehandle
```

Operace *otevření souboru* nám zpřístupní data v souboru přes objekt, kterému se říká *file handle*. Může to být interně obyčejné číslo nebo ukazatel, kterým potom pro systém identifikuje soubor, se kterým chceme pracovat. Souborů můžeme otevřít víc najednou, přičemž každý bude mít svůj vlastní *handle*. Protože ale každé otevření souboru s sebou nese určitou režii v paměti, operační systém většinou má nějaký limit, kolik souborů může jeden proces otevřít. V GNU/Linux tento limit zjistíme nebo nastavíme pomocí příkazu `ulimit -n`.

```bash
$ ulimit -n
1024
```

Zavření souboru je vhodné provádět v okamžiku, kdy práci se souborem končíme. Není dobré se spoléhat na to, že se zavření souboru provede automaticky! Data, která zapisujeme do souboru se nezapisují okamžitě, ale po určitých blocích. Může se tedy stát, že se např. program předčasně ukončí s chybou a nezavřený soubor se nezapíše na disk. Stejně tak, pokud pracujeme s velkým množstvím souborů, tak může být nutné explicitně soubory zavírat, abychom se nedostali nad limit povoleného počtu otevřených souborů.

### Práce se soubory v Pythonu

Základní idiom pro práci se soubory v Pythonu je následující:

```python
with open(filename, mode) as handle:
    do_something_with_handle
```

Tento tzv. kontextový manažer zajistí, že na konci bloku bude soubor uzavřen i pokud dojde někde během provádění bloku k chybě. Proměnná `handle` je objekt, kterým k souboru přistupujeme. Ten lze použít např. jako argument `file` pro funkci `print`, která místo na "obrazovku" bude vypisovat do souboru.

Argument `filename` určuje jméno souboru (může být i s celou cestou) a `mode` je mód otevření: `r`=pro čtení, `w`=pro zápis. Pozor při otevírání pro zápis, pokud už soubor existuje, tak se jeho obsah bez ptaní přepíše! Pro kontrolu existence souboru před přepsáním můžeme použít mód `x`, pro zápis na konec existujícího souboru mód `a`. Čtení a zápis zároveň umí mód `+`.

In [ ]:
numerals = '0123456789abcdefghijklmnopqrsuvwxyz'

def num2str(n, base=2):
    if n == 0:
        return '0'

    ret = ''
    while n:
        rem = n % base
        n = n // base
        ret = numerals[rem] + ret

    return ret


with open('base-conversion.csv', 'w') as outfile:
    print('original,base2,base10,base16,base35', file=outfile)
    for i in range(256):
        print(i, num2str(i, 2), num2str(i, 10), num2str(i, 16), num2str(i, 35), sep=',', file=outfile)
        # or...
        # outfile.write(f'{i},{num2str(i, 2)},{num2str(i, 10)},{num2str(i, 16)},{num2str(i, 35)}\n')

Místo funkce `print` můžeme také použít metodu `write` handle-objektu. Tady ale musíme použít objektovou syntaxi volání `handle.write()`.

Bez kontextového manažeru bychom mohli kód zapsat ekvivalentně nějak takto:

In [ ]:
outfile = open('base-conversion.csv', 'w')
print('original,base2,base10,base16,base35', file=outfile)
for i in range(256):
    print(i, num2str(i, 2), num2str(i, 10), num2str(i, 16), num2str(i, 35), sep=',', file=outfile)
    
outfile.close()

*Pozor ale na rozdíl v zavírání souboru! Kontextový manažer zajistí, že se soubor zavře, i pokud dojde při běhu programu k chybě. To při "ruční" práci se soubory bychom museli zajistit sami odchytáváním potenciálních chyb.*

Čtení souboru funguje podobně, jen místo metody `write` použijeme některou z metod `read` (načti celý soubor nebo určitý počet znaků jako jeden dlouhý řetězec), `readline` (načti jeden řádek), `readlines` (načti řádky jako *seznam* řetězců). Objekt `handle` se také chová jako iterátor, můžeme tedy rovnou ve `for` cyklu načítat řádky iterací přes `handle`. V závislosti na tom, co chceme, může čtení souboru vypadat třeba následovně:

In [ ]:
with open('base-conversion.csv', 'r') as infile:
    header = infile.readline()
    print(header, end='')
    for i in infile:
        print(i.strip())

Ve výchozím stavu předpokládá Python, že otevíráme textový soubor a postará se o dekódování na základě lokalizace nastavené uživatelem nebo v operačním systému. Může se ale stát, že čteme soubor v jiném kódování než byl zapsán, potom můžeme funkci `open` parametrem `encoding` signalizovat, že je potřeba provést překódování. Pamatujme si přitom, že v Pythonu (od verze 3) je každý řetězec interně Unicode...

In [ ]:
with open('text-utf8.txt', 'r', encoding='utf-8') as infile:
    print(mytext:=infile.readline())

In [ ]:
with open('text-cp1250.txt', 'w', encoding='cp1250') as outfile:
    outfile.write(mytext)

##### CSV soubory

Oblíbené textové CSV (comma-separated values) soubory podporuje řada knihoven, počínaje vestavěnou `csv`, ale taky `numpy` nebo `pandas`. Čtení CSV souborů je potom hračka...

In [ ]:
from functools import partial
import numpy as np

conv_data = np.genfromtxt('base-conversion.csv', delimiter=',',
                          names=True, usecols=(0, 1),
                          converters={1: partial(int, base=2)},
                          dtype=(int, int))
print(conv_data)

### Binární soubory

Přestože technicky vzato jsou všechny soubory binární (text je také nějak binárně kódován), rozlišujeme při programování textové a binární soubory ve smyslu, že ty první interpretujeme jako text a ty druhé jako jiná data. Číselná data můžeme ukládat v obou typech souborů, ale každý způsob má své výhody a nevýhody. Čísla uložená jako text jsou pohodlná pro uživatele, ale pro počítač náročnější na zpracování. Binární data přesně naopak, pro uživatele jsou obtížně čitelná, ale často zabírají méně místa a jejich zpracování je rychlejší.

Pro ukládání dat do souborů existuje mnoho různých formátů a často existuje knihovna, která takový formát umí číst a zapisovat. Ve vědeckém světě jsou to například: `HDF`, `NetCDF`, `FITS`, `Silo`, `.root` a další. Například soubory ve formátu `NetCDF` podporují knihovny `netCDF4`, `xarray`, `iris` a další.

In [ ]:
from netCDF4 import Dataset as ncfile
import matplotlib.pyplot as plt

infile = ncfile('geo_em.d01.nc')
print(infile.dimensions)
print(infile.variables)

plt.pcolormesh(infile.variables['XLONG_M'][0], infile.variables['XLAT_M'][0], infile.variables['HGT_M'][0])
plt.savefig('geo_em.d01.png')

Pokud nemáme to štěstí a knihovna neexistuje, můžeme si vypomoct obecnými funkcemi pro konverzi Pythonovských objektů do binární reprezentace, například v knihovně `struct`. Pro zakódování do binární reprezentace použijeme funkci `pack`:

In [ ]:
import struct

# Little-endian 4B integer
print(struct.pack('<i', 1))
# Six big-endian 4B integers
print(struct.pack('>6i', 1, 12, 234, 357, 987, 274892375))
# Little-endian 4B float
print(struct.pack('<f', 3.45e-12).hex())
# Big-endian 8B float
print(struct.pack('>d', 3.45e-67).hex())

Tento binární "řetězec" potom můžeme uložit přímo do souboru.

In [ ]:
with open('bindata.dat', 'wb') as outfile:
    outfile.write(struct.pack('<6i', 1, 12, 234, 357, 987, 274892375))

Naopak po načtení binárních dat z nich můžeme Pythonovské objekty udělat funkcí `unpack`.

In [ ]:
with open('bindata.dat', 'rb') as infile:
    data = np.array(struct.unpack('<6i', infile.read()))
    print(data)

*K zamyšlení: co dostaneme, pokud celá čísla zapsaná do souboru přečteme jako floaty?*

In [ ]:
with open('bindata.dat', 'rb') as infile:
    data = np.array(struct.unpack('<6f', infile.read()))
    # data = np.array(struct.unpack('<6f', infile.read()), dtype=np.float32)
    print(data)

#### I/O "rychle a levně"

Někdy nepotřebujeme data ukládat v nějakém specifickém formátu, ale postačí nám je jakkoliv uložit pro pozdější načtení. Na toto nám často postačí vestavěná knihovna `pickle`...

In [ ]:
import pickle

with open('data.pickle', 'wb') as outfile:
    pickle.dump(data, outfile)

with open('data.pickle', 'rb') as infile:
    data_in = pickle.load(infile)
    print(data_in)

Pokud preferujeme textový formát dat a naše datové struktury nejsou příliš "zvláštní", můžeme použít např. formát json podporovaný knihovnou `json`.

In [ ]:
import json

with open('data.json', 'r') as infile:
    data = json.load(infile)
    print(data)

In [ ]:
del data['Af']
with open('data2.json', 'w') as outfile:
    json.dump(data, outfile)

### Práce se souborovým systémem

Operační systémy nám poskytují služby pro práci se soubory, které odstiňují technikálie jako např. jak je soubor uložený na fyzickém médiu, kde ho na médiu najdeme atd. Bohužel, během historického vývoje se různé OS neshodly na tom, jak budou soubory uživateli zpřístupňovat "logicky". I když se většinou shodnou na základním dělení (adresáře->soubory), nomenklatura se často zásadně liší. Musíme proto brát v úvahu zvláštnosti jednotlivých světů. Na unixových OS například se vše tváří jako *soubor* (adresář je speciální soubor typu *directory*), v názvech souborů se rozlišují velká a malá písmena, oddělovač adresářů je lomítko (/) a přípony souborů nemají pro OS speciální význam. Naproti tomu u druhého dominantního hráče, tedy MS Windows, cesta k souboru začíná označením disku (např. C:), oddělovačem je opačné lomítko (\\) a přípona souboru určuje jeho typ.

Př:
```
Unix:    /home/jose/Downloads/soubor.pdf
Windows: C:\Users\jose\Downloads\soubor.pdf
```

Ve standardní knihovně Pythonu existují funkce, které tyto rozdíly můžou pomoci překonat tak, abychom mohli psát jednotný kód, který potom poběží na "Windows" i na "Linuxu". Hlavní pomůckou je zde modul `os.path` a funkce jako `basename`, `join`, `split`, `splitext`, `splitdrive` apod.

*Na Windows můžeme s výhodou použít raw string, abychom nemuseli všude zdvojovat opačné lomítko...*

In [ ]:
import os.path

filepath = r'C:\Users\jose\Downloads\soubor.pdf'
print(os.path.split(filepath))
print(os.path.splitdrive(filepath))
print(os.path.splitext(filepath))

In [ ]:
print(os.path.join('C:', 'Users', "jose", "Downloads", "soubor.pdf"))

# na unix-like OS by to bylo něco jako:
# print(os.path.join('/home', "jose", "Downloads", "soubor.pdf"))

Z dalších zajímavých knihoven pro práci se soubory doporučuji se podívat na `glob` a `shutil`.

Ukázka použití knihovny `glob` pro vypsání seznamu souborů odpovídajících danému vzoru (podle tzv. wildcards), např. všechny "soubory s příponou .txt":

In [ ]:
import glob

for i in glob.glob('*.txt'):
    print(i)

### Trik pro přátele příkazové řádky

Spouštění programů v příkazové řádce nám umožňuje použít jednoduchou formu I/O, kdy z programu necháme výstup vypsat na obrazovku např. funkcí `print`, ale necháme ho operační systém *přesměrovat* do souboru.

```bash
python program.py > vystup.txt
```

Stejně tak pro čtení ze souboru můžeme v programu použít funkci `input` pro čtení z klávesnice, ale na příkazové řádce *přesměrovat* vstup ze souboru.

```bash
python program.py < vstup.txt
```

### Vizualizace

Jedním z častých typů "výstupu" z programu je graf, obrázek nebo jiný způsob vizualizace. Viděli jsme už pár příkladů použití knihovny `matplotlib` pro vykreslení grafů, ale pokud nechceme kreslit přímo z Pythonu, můžeme využít výstup do souboru, který potom načteme v nějakém jiném softwaru. Oblíbený je v tomto formát CSV, který lze načíst pomocí různých programů včetně *MS Excel* nebo jiného spreadsheetu, *gnuplotu* a dalších.

In [ ]:
def leibniz(n):
    sums = []
    running_sum = 0
    for k in range(n):
        running_sum += (-1)**k/(2*k+1)
        sums.append(running_sum*4)

    return sums

leibniz_sums = leibniz(1_000)
with open('leibniz.csv', 'w') as outfile:
    for k, v in enumerate(leibniz_sums):
        print(k, v, sep=',', file=outfile)

Nebo můžeme využít knihovny jako `matplotlib` nebo jiné (`seaborn`, `plotly`, `vispy` a mnoho dalších) a kreslit přímo z Pythonu.

In [ ]:
import math
import matplotlib.pyplot as plt

leibniz_sums = leibniz(10_000)

fig, ax = plt.subplots()
ax.axhline(y=math.pi, color='red')
ax.plot(leibniz_sums, marker='o', markersize=1, color='green', linestyle='', alpha=0.5)
ax.set_xlim(0, len(leibniz_sums))
ax.set_ylim(3.14, 3.143)
ax.set_xlabel('$k$')
ax.set_ylabel(r'$\pi$ approximation')
ax.set_title('Leibniz formula')

# plt.show()
plt.savefig('leibniz.png')

#### Další trik pro přátele příkazové řádky

V unixovém světě, kde se programy často spouští z příkazové řádky, je oblíbeným způsobem práce parametrizace chování programu argumenty zadanými při spouštění. Při práci se soubory toho můžeme s výhodou využít a (nejen) jméno vstupního nebo výstupního souboru zadat programu při spuštění. V jednoduchých případech stačí využít seznamu argumentů `argv` přístupného v knihovně `sys`.

```python
import sys

def leibniz(n):
    sums = []
    running_sum = 0
    for k in range(n):
        running_sum += (-1)**k/(2*k+1)
        sums.append(running_sum*4)

    return sums


if __name__ == '__main__':
    leibniz_sums = leibniz(int(sys.argv[2]))
    with open(sys.argv[1], 'w') as outfile:
        for k, v in enumerate(leibniz_sums):
            print(k, v, sep=',', file=outfile)
```

Pro komplexnější zpracování argumentů je vhodnější použít např. knihovnu `argparse`.

```python
from argparse import ArgumentParser

def leibniz(n):
    sums = []
    running_sum = 0
    for k in range(n):
        running_sum += (-1)**k/(2*k+1)
        sums.append(running_sum*4)

    return sums


if __name__ == '__main__':
    argp = ArgumentParser()
    argp.add_argument('outfile')
    argp.add_argument('-m', '--max_k', default=1000, type=int)
    argp.add_argument('-s', '--separator', default=',')
    argp.add_argument('-d', '--decimal_separator', default='.')
    argv = argp.parse_args()

    leibniz_sums = leibniz(argv.max_k)
    with open(argv.outfile, 'w') as outfile:
        for k, v in enumerate(leibniz_sums):
            print(k, str(v).replace('.', argv.decimal_separator),
                  sep=argv.separator, file=outfile)
```

Konstrukce `if __name__ == '__main__':` umožňuje, abychom skript mohli spouštět zároveň jako program i použít jako knihovnu, přičemž kód uvedený za touto podmínkou se spustí pouze v případě, kdy skript spouštíme jako program. Funkci `leibniz` ale můžeme naimportovat i do jiného skriptu. V tu chvíli se kód za podmínkou `if __name__ == '__main__':` nespustí.

In [ ]:
from leibniz import leibniz

print(leibniz(10))